In [1]:
import numpy as np 
import quadprog
import matplotlib.pyplot as plt 
import pandas as pd
import requests
import ast
import time
import schedule
from datetime import date
import json
from datetime import datetime
from influxdb import InfluxDBClient

In [2]:
client = InfluxDBClient(host='ec2-13-58-107-29.us-east-2.compute.amazonaws.com',username='ewre_student',password='happylife!',database='tomorrow_api' )

In [3]:
import requests

url = "https://api.tomorrow.io/v4/weather/forecast?location=42.3478%2C%20-71.0466&fields=temperature&units=metric&timesteps=1m&startTime=now&endTime=nowPlus6h&apikey=VYLHerX2Zbk6nBvVo4VTkpGg2JFJTfw3"

headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip"
}

response = requests.get(url, headers=headers)


In [4]:
location=[('location', '30.28412, -97.73118'), #PRC
('location', '30.3265250, -97.7199113'),#Relly pond
('location', '30.2871667, -97.7341111')]#bridge5

In [5]:
def collect_tomorrow_API_forecast_data(i):
    set_location=location[i]

    params = (set_location,('units', 'imperial'),('startTime','now'),('endTime','nowPlus2h'),('apikey', 'VYLHerX2Zbk6nBvVo4VTkpGg2JFJTfw3'),)
    headers = {
    "Accept": "application/json",
    "Accept-Encoding": "gzip"}

    response = requests.get('https://api.tomorrow.io/v4/weather/forecast?', params=params, headers=headers)
    time.sleep(10)

    #change to dictionary format
    content=response.content
    dict_str = content.decode("ascii")
    dict_str
    my_data = json.loads(dict_str)
    value=my_data['timelines']['minutely']


    _location=location[i][1]
    _measurement='weather_forecast'
    json_body = []
    for i in range(len(value)):
        current_time=value[i]['time']
        rain_probability=value[i]['values']['precipitationProbability']
        temp=value[i]['values']['temperature']
        rain_intensity=value[i]['values']['rainIntensity']

        json_data={
                        "measurement": _measurement,
                        "tags": {
                            "location":_location
                        },
                        "time": current_time,
                        "fields": {
                            "precipitationProbability": float(rain_probability),
                            "temperature": float(temp),
                            "rainIntensity": float(rain_intensity)

                        }
                    },
        json_body.extend(json_data)
    client.write_points(json_body)






In [6]:
#Tomorrow.io API https://docs.tomorrow.io/reference/get-timelines
# use schedule module to do work


def crawling_job():
    for i in range(0,len(location)):
        collect_tomorrow_API_forecast_data(i)

     
                        
            

In [ ]:

#Adaptive crawling -pretest for collecting date 
schedule.every().minute.at(":15").do(crawling_job)
while True:
    schedule.run_pending()
    time.sleep(10)
       